# Smoothing solution to the MSD 

In [37]:
import gtsam
from typing import Optional, List
from functools import partial
from gtsam.symbol_shorthand import X
import numpy as np
import graphviz
import matplotlib.pyplot as plt

# Create factors functions

In [38]:
m = 1.0
k = 10.0
c = 1.0

A = np.array([[0, 1],
             [-k/m, -c/m]])
B = np.array([[0],
             [1/m]])
DT = 0.01

def error_odom(f_nom: np.ndarray, this: gtsam.CustomFactor,
               values: gtsam.Values,
               jacobians: Optional[List[np.ndarray]]) -> float:
    """Odometry Factor error function
    :param measurement: Odometry measurement, to be filled with `partial`
    :param this: gtsam.CustomFactor handle
    :param values: gtsam.Values
    :param jacobians: Optional list of Jacobians
    :return: the unwhitened error
    """
    key1 = this.keys()[0]
    key2 = this.keys()[1]
    q1 = values.atVector(key1).reshape(-1,1) 
    q2 = values.atVector(key2).reshape(-1,1)

    qdot1 = A @ q1 + B * f_nom
    q2_predict = q1 + qdot1 * DT

    error = q2 - q2_predict

    #jacobians of h
    if jacobians is not None:
        jacobians[0] = - (np.eye(2) + A * DT)
        jacobians[1] =  np.eye(2)

    return error

def error_z(measurement: np.ndarray, this: gtsam.CustomFactor,
              values: gtsam.Values,
              jacobians: Optional[List[np.ndarray]]) -> float:
    key = this.keys()[0]
    estimate = values.atVector(key)[0]
    error = estimate - measurement

    if jacobians is not None:
        jacobians[0] = np.array([[1.0, 0.0]])

    return error

# Constants

In [39]:
COV_R = np.eye(2) * 1e-2
COV_Q = np.array([[0.05**2]]) * 1e1
COV_PRIOR = np.eye(2) * 1e-15

GTSAM_COV_PRIOR = gtsam.noiseModel.Gaussian.Covariance(COV_PRIOR)
GTSAM_COV_R = gtsam.noiseModel.Gaussian.Covariance(COV_R)
GTSAM_COV_Q = gtsam.noiseModel.Gaussian.Covariance(COV_Q)

# Simulation for data gathering

In [40]:
#INITALIZE
t_0 = 0
t_final = 10
t_vec =  np.arange(t_0, t_final, DT)

q = np.array([1,1]).reshape(-1,1)
history = {"gt": [q], "u": [], "z": []}

f_nom = 1.0
#SIMULATION
for t in t_vec[1:]:
    f_real = np.random.normal(f_nom,3)

    qdot = A @ q + B * f_real
    q = q + qdot * DT
    z = np.random.normal(q[0],0.05)

    history["u"].append(f_nom)
    history["z"].append(z)
    history["gt"].append(q)

# Building the factor graph

In [41]:
graph = gtsam.NonlinearFactorGraph()

for i, (u_i, z_ip1) in enumerate(zip(history["u"],history["z"])):
    graph.add(gtsam.CustomFactor(GTSAM_COV_R, 
                                 [X(i), X(i+1)], 
                                 partial(error_odom, u_i)))
    graph.add(gtsam.CustomFactor(GTSAM_COV_Q, 
                                 [X(i+1)], 
                                 partial(error_z, z_ip1)))

In [42]:
graph.saveGraph("graph_MSD")
graphviz.render('dot','pdf',"graph_MSD") #creates PDF

'graph_MSD.pdf'

# Initalizing inital values for solver

In [43]:
q_dr = history["gt"][0]
history["dr"] = [q_dr] #add dead reckoning to history dictionary for later plots

initial_estimate = gtsam.Values()
initial_estimate.insert(X(0), q_dr)

for i, u_i in enumerate(history["u"]):
    qdot_dr = A @ q_dr + B * f_nom
    q_dr = q_dr + qdot_dr * DT
    
    initial_estimate.insert(X(i+1), q_dr)

    history["dr"].append(q_dr)

# Solving with the optimizer

In [44]:
params = gtsam.GaussNewtonParams()
optimizer = gtsam.GaussNewtonOptimizer(graph, initial_estimate,params)

result = optimizer.optimize()
marginals = gtsam.Marginals(graph, result)

# Plot

In [45]:
# %matplotlib inline
%matplotlib auto

gt = np.hstack([_[0] for _ in history["gt"]])
ests = np.array([result.atVector(X(i))[0] for i in range(len(t_vec))])
stds = np.array([np.sqrt(marginals.marginalCovariance(X(i))[0,0]) for i in range(len(t_vec))])
z = np.hstack([np.nan] + history["z"])

plt.figure()
graphics_est, = plt.plot(t_vec, ests, color = 'black')
plt.plot(t_vec, ests + 1 *stds, color = 'black')
plt.plot(t_vec, ests - 1 *stds, color = 'black')
graphics_z = plt.scatter(t_vec, z, s = 1, color = 'red')
graphics_gt, = plt.plot(t_vec, gt, color = 'blue')
plt.legend([graphics_gt, graphics_est, graphics_z], ['x_gt', 'x_est', 'z'])
plt.title('factor graph smoothing')
plt.show()


Using matplotlib backend: TkAgg


In [49]:
np.array(z)

/tmp/ipykernel_481763/730665556.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(z)


array([nan, array([0.97042664]), array([0.98121627]), array([1.01196764]),
       array([1.07302424]), array([1.05163526]), array([1.04081681]),
       array([0.99368704]), array([0.97527251]), array([1.01391927]),
       array([1.02986515]), array([1.04711502]), array([1.14136238]),
       array([1.03599285]), array([1.13873459]), array([1.07161487]),
       array([1.13968752]), array([0.99464305]), array([1.03531832]),
       array([0.88790869]), array([1.07367282]), array([0.94688558]),
       array([1.02137733]), array([0.90655576]), array([0.92281881]),
       array([0.98114731]), array([0.90525672]), array([0.89909324]),
       array([0.9235695]), array([0.8962037]), array([0.91413112]),
       array([0.85429141]), array([0.78870262]), array([0.84649917]),
       array([0.76948451]), array([0.74588904]), array([0.76991594]),
       array([0.68152252]), array([0.63289678]), array([0.71540663]),
       array([0.56595877]), array([0.6109159]), array([0.67369011]),
       array([0.48

In [46]:
gt = [_[0] for _ in history["gt"]]